In [1]:
#import tensorflow as tf
import numpy as np
import pandas as pd
import os
from keras import backend as K 
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint
from keras.models import load_model, Model
from keras.layers import GlobalAveragePooling2D, Dense
#from keras.layers import GlobalAveragePooling2D, Dropout, Dense
from keras.applications import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from sklearn.model_selection import KFold

Using TensorFlow backend.


In [2]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [3]:
train_data = pd.read_csv('/media/7D63-84F2/data/ISIC2018_Task3_Training_GroundTruth_new.csv')

Y = train_data[['label']]

kf = KFold(n_splits = 5)

VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

In [4]:
image_dir = '/media/7D63-84F2/data/ISIC2018_Task3_Training_Input/'
save_dir = './saved_models/'
fold_var = 1

In [5]:
idg = image.ImageDataGenerator(preprocessing_function=preprocess_input,
                        rotation_range=40,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        zoom_range=0.2,
                        shear_range=0.2,
                        fill_mode='nearest',
                        horizontal_flip = True,
                        rescale=1./255)

In [6]:
for train_index, val_index in kf.split(np.zeros(10015),Y):
    training_data = train_data.iloc[train_index]
    validation_data = train_data.iloc[val_index]

    train_data_generator = idg.flow_from_dataframe(training_data,
                                                   directory = image_dir,
                                                   x_col = "filename",
                                                   y_col = "label",
                                                   target_size=(75, 75),
                                                   batch_size=8,
                                                   color_mode="rgb",
                                                   class_mode = "categorical",
                                                   shuffle = True)
    
    valid_data_generator  = idg.flow_from_dataframe(validation_data,
                                                    directory = image_dir,
                                                    x_col = "filename",
                                                    y_col = "label",
                                                    target_size=(75, 75),
                                                    batch_size=8,
                                                    color_mode="rgb",
                                                    class_mode = "categorical",
                                                    shuffle = True)

    # CREATE NEW MODEL
    base_model = InceptionV3(weights='imagenet', include_top=False)
    CLASSES = 7
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    # x = Dropout(0.4)(x)
    predictions = Dense(CLASSES, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    for layer in base_model.layers:
        layer.trainable = False
    
    # COMPILE NEW MODEL
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    # CREATE CALLBACKS
    checkpoint = ModelCheckpoint(save_dir+get_model_name(fold_var), 
                            monitor='val_accuracy', verbose=1, 
                            save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    # There can be other callbacks, but just showing one because it involves the model name
    # This saves the best model
    # FIT THE MODEL
    
    EPOCHS = 5
    #STEPS_PER_EPOCH = 126
    #steps_per_epoch=STEPS_PER_EPOCH,
    history = model.fit_generator(
        train_data_generator,
        epochs=EPOCHS,
        verbose=1,
        callbacks=callbacks_list,
        validation_data=valid_data_generator)
 
    #PLOT HISTORY
    #		:
    #		:

    # LOAD BEST MODEL to evaluate the performance of the model
    model.load_weights("./saved_models/model_"+str(fold_var)+".h5")

    results = model.evaluate(valid_data_generator)
    results = dict(zip(model.metrics_names,results))

    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])

    K.clear_session()
    
    fold_var += 1

Found 8012 validated image filenames belonging to 7 classes.
Found 2003 validated image filenames belonging to 7 classes.
Epoch 1/5
1002/1002 [==============================] - 389s 388ms/step - loss: 1.2745 - accuracy: 0.6271 - val_loss: 3.7283 - val_accuracy: 0.6860

Epoch 00001: val_accuracy improved from -inf to 0.68597, saving model to ./saved_models/model_1.h5
Epoch 2/5
1002/1002 [==============================] - 336s 336ms/step - loss: 1.2335 - accuracy: 0.6278 - val_loss: 9.6664 - val_accuracy: 0.6860

Epoch 00002: val_accuracy did not improve from 0.68597
Epoch 3/5
1002/1002 [==============================] - 330s 330ms/step - loss: 1.2542 - accuracy: 0.6226 - val_loss: 0.0462 - val_accuracy: 0.6860

Epoch 00003: val_accuracy did not improve from 0.68597
Epoch 4/5
1002/1002 [==============================] - 330s 330ms/step - loss: 1.2517 - accuracy: 0.6264 - val_loss: 3.4021 - val_accuracy: 0.6860

Epoch 00004: val_accuracy did not improve from 0.68597
Epoch 5/5
1002/1002 [=

In [7]:
VALIDATION_ACCURACY

[0.6859710216522217,
 0.6874687671661377,
 0.6849725246429443,
 0.6969545483589172,
 0.5921118259429932]

In [8]:
VALIDATION_LOSS

[5.954578876495361,
 2.9831764698028564,
 0.006127266678959131,
 3.0566112995147705,
 6.360202312469482]

In [12]:
with open('resultsfile.txt', 'w') as f:
    f.write('Validation Accuracy\n')
    f.writelines("%s, " % num for num in VALIDATION_ACCURACY)
    f.write('\nValidation Loss\n')
    f.writelines("%s, " % num for num in VALIDATION_LOSS)